In [1]:
import geopandas as gpd
import os
import fiona
import ipywidgets as widgets
from IPython.display import display
from rasterstats import zonal_stats
import rasterio
import rasterio.fill
from shapely.geometry import shape, mapping
import json
from earthpy import clip
import earthpy.spatial as es
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
import gdal
import datetime
import warnings
import pandas as pd
import scipy.spatial
warnings.filterwarnings('ignore')

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

''

# Processing Rasters

In [2]:
def processing_raster(name, method, clusters):
    messagebox.showinfo('OnSSET', 'Select the ' + name + ' map')
    raster=rasterio.open(filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*"))))
    
    clusters = zonal_stats(
        clusters,
        raster.name,
        stats=[method],
        prefix=name, geojson_out=True, all_touched=True)
    
    print(datetime.datetime.now())
    return clusters

# Processing Elevation and Slope

In [3]:
def processing_elevation_and_slope(name, method, clusters, workspace,crs):
    messagebox.showinfo('OnSSET', 'Select the ' + name + ' map')
    raster=rasterio.open(filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*"))))
    
    clusters = zonal_stats(
        clusters,
        raster.name,
        stats=[method],
        prefix=name, geojson_out=True, all_touched=True)

    gdal.Warp(workspace + r"\dem.tif",raster.name,dstSRS=crs)

    def calculate_slope(DEM):
        gdal.DEMProcessing(workspace + r'\slope.tif', DEM, 'slope')
        with rasterio.open(workspace + r'\slope.tif') as dataset:
            slope=dataset.read(1)
        return slope

    slope=calculate_slope(workspace + r"\dem.tif")

    slope = rasterio.open(workspace + r'\slope.tif')
    gdal.Warp(workspace + r'\slope_4326.tif',slope.name,dstSRS='EPSG:4326')
    slope_4326 = rasterio.open(workspace + r'\slope_4326.tif')

    clusters = zonal_stats(
        clusters,
        slope_4326.name,
        stats=["majority"],
        prefix="sl_", all_touched = True, geojson_out=True)
    
    print(datetime.datetime.now())
    return clusters

# Finalizing rasters

In [4]:
def finalizing_rasters(workspace, clusters, crs):
    clusters["Placeholder"] = 0
    output = workspace + r'\placeholder.geojson'
    with open(output, "w") as dst:
        collection = {
            "type": "FeatureCollection",
            "features": list(clusters)}
        dst.write(json.dumps(collection))
  
    clusters = gpd.read_file(output)
    os.remove(output)
    
    print(datetime.datetime.now())
    return clusters

# Preparing for vectors

In [5]:
def preparing_for_vectors(workspace, clusters, crs):   
    clusters.crs = {'init' :'epsg:4326'}
    clusters = clusters.to_crs({ 'init': crs}) 
    points = clusters.copy()
    points["geometry"] = points["geometry"].centroid
    points.to_file(workspace + r'\clusters_cp.shp', driver='ESRI Shapefile')
    print(datetime.datetime.now())    
    return clusters

# Processing Lines

In [6]:
def processing_lines(name, admin, crs, workspace, clusters):
    messagebox.showinfo('OnSSET', 'Select the ' + name + ' map')
    #lines_f=fiona.open(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))
    lines=gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))

    lines_clip = clip.clip_shp(lines, admin)
    lines_clip.crs = {'init' :'epsg:4326'}
    lines_proj=lines_clip.to_crs({ 'init': crs})

    lines_proj.to_file(workspace + r"\ " + name + "_proj.shp", driver='ESRI Shapefile')

    line = fiona.open(workspace +  r"\ " + name + "_proj.shp")
    firstline = line.next()

    schema = {'geometry' : 'Point', 'properties' : {'id' : 'int'},}
    with fiona.open(workspace + r"\ " + name + "_proj_points.shp", "w", "ESRI Shapefile", schema) as output:
        for lines in line:
            if lines["geometry"] is not None:
                first = shape(lines['geometry'])
                length = first.length
                for distance in range(0,int(length),100):
                    point = first.interpolate(distance)
                    output.write({'geometry' :mapping(point), 'properties' : {'id':1}})

    lines_f = fiona.open(workspace + r"\ " + name + "_proj_points.shp")
    lines = gpd.read_file(workspace +  r"\ " + name + "_proj.shp")
    points = fiona.open(workspace + r'\clusters_cp.shp')

    geoms1 = [shape(feat["geometry"]) for feat in lines_f]
    s1 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms1]
    s1_arr = np.array(s1)

    geoms2 = [shape(feat["geometry"]) for feat in points]
    s2 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms2]
    s2_arr = np.array(s2)

    def do_kdtree(combined_x_y_arrays,points):
        mytree = scipy.spatial.cKDTree(combined_x_y_arrays)
        dist, indexes = mytree.query(points)
        return dist, indexes

    def vector_overlap(vec, settlementfile, column_name):
        vec.drop(vec.columns.difference(["geometry"]), 1, inplace=True)
        a = gpd.sjoin(settlementfile, vec, op = 'intersects')
        a[column_name + '2'] = 0
        return a  

    results1, results2 = do_kdtree(s1_arr,s2_arr)

    z=results1.tolist()
    clusters[name+'Dist'] = z
    clusters[name+'Dist'] = clusters[name+'Dist']/1000

    a = vector_overlap(lines, clusters, name+'Dist')

    clusters = pd.merge(left = clusters, right = a[['id',name+'Dist2']], on='id', how = 'left')
    clusters.drop_duplicates(subset ="id", keep = "first", inplace = True) 

    clusters.loc[clusters[name+'Dist2'] == 0, name+'Dist'] = 0

    del clusters[name+'Dist2']
    print(datetime.datetime.now())
    return clusters

# Processing points


In [1]:
def processing_points(name, admin, crs, workspace, clusters):
    messagebox.showinfo('OnSSET', 'Select the ' + name + ' map')
    #points_f=fiona.open(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))
    points=gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))

    points_clip = clip.clip_shp(points, admin)
    points_clip.crs = {'init' :'epsg:4326'}
    points_proj=points_clip.to_crs({ 'init': crs})

    points_proj.to_file(workspace + r"\ " + name + "_proj.shp", driver='ESRI Shapefile')

    points_f = fiona.open(workspace + r"\ " + name + "_proj.shp")
    points = gpd.read_file(workspace +  r"\ " + name + "_proj.shp")
    points2 = fiona.open(workspace + r'\clusters_cp.shp')

    geoms1 = [shape(feat["geometry"]) for feat in points_f]
    s1 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms1]
    s1_arr = np.array(s1)
    
    geoms2 = [shape(feat["geometry"]) for feat in points2]
    s2 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms2]
    s2_arr = np.array(s2)

    def do_kdtree(combined_x_y_arrays,points):
        mytree = scipy.spatial.cKDTree(combined_x_y_arrays)
        dist, indexes = mytree.query(points)
        return dist, indexes

    def vector_overlap(vec, settlementfile, column_name):
        vec.drop(vec.columns.difference(["geometry"]), 1, inplace=True)
        a = gpd.sjoin(settlementfile, vec, op = 'intersects')
        a[column_name + '2'] = 0
        return a  

    results1, results2 = do_kdtree(s1_arr,s2_arr)

    z=results1.tolist()
    clusters[name+'Dist'] = z
    clusters[name+'Dist'] = clusters[name+'Dist']/1000

    a = vector_overlap(points, clusters, name+'Dist')

    clusters = pd.merge(left = clusters, right = a[['id',name+'Dist2']], on='id', how = 'left')
    clusters.drop_duplicates(subset ="id", keep = "first", inplace = True) 

    clusters.loc[clusters[name+'Dist2'] == 0, name+'Dist'] = 0

    del clusters[name+'Dist2']
    print(datetime.datetime.now())
    return clusters

# Processing hydro

In [2]:
def processing_hydro(name, admin, crs, workspace, clusters, points):

    points_clip = clip.clip_shp(points, admin)
    points_clip.crs = {'init' :'epsg:4326'}
    points_proj=points_clip.to_crs({ 'init': crs})

    points_proj.to_file(workspace + r"\ " + name + "_proj.shp", driver='ESRI Shapefile')

    points_f = fiona.open(workspace + r"\ " + name + "_proj.shp")
    points = gpd.read_file(workspace +  r"\ " + name + "_proj.shp")
    points2 = fiona.open(workspace + r'\clusters_cp.shp')

    #geoms1 = [shape(feat["geometry"]) for feat in points_f]
    #s1 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms1]
    #s1_arr = np.array(s1)
    #s1_arr = points.to_numpy()
    
    
    geoms1 = []
    for each in points_f:
        geoms1.append(each["properties"])
        geoms1.append(each["geometry"])
    
    s1_arr = np.array(geoms1)
    
    geoms2 = [shape(feat["geometry"]) for feat in points2]
    s2 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms2]
    s2_arr = np.array(s2)

    def do_kdtree(combined_x_y_arrays,points):
        mytree = scipy.spatial.cKDTree(combined_x_y_arrays)
        dist, indexes = mytree.query(points)
        return dist, indexes

    def vector_overlap(vec, settlementfile, column_name):
        vec.drop(vec.columns.difference(["geometry"]), 1, inplace=True)
        a = gpd.sjoin(settlementfile, vec, op = 'intersects')
        a[column_name + '2'] = 0
        return a  

    results1, results2 = do_kdtree(s1_arr,s2_arr)

    z=results1.tolist()
    clusters[name+'Dist'] = z
    clusters[name+'Dist'] = clusters[name+'Dist']/1000

    a = vector_overlap(points, clusters, name+'Dist')

    clusters = pd.merge(left = clusters, right = a[['id',name+'Dist2']], on='id', how = 'left')
    clusters.drop_duplicates(subset ="id", keep = "first", inplace = True) 

    clusters.loc[clusters[name+'Dist2'] == 0, name+'Dist'] = 0

    del clusters[name+'Dist2']
    print(datetime.datetime.now())
    return clusters
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    #from shapely.ops import nearest_points

    #hydro_clip = clip.clip_shp(hydro, admin)
    #hydro.crs = {'init' :'epsg:4326'}
    #hydro_proj=hydro.to_crs({ 'init': crs})

    #hydro_proj.to_file(workspace + r"\ " + name + "_proj.shp", driver='ESRI Shapefile')

    #points = gpd.read_file(workspace +  r"\ " + name + "_proj.shp")
    #points2 = fiona.open(workspace + r'\clusters_cp.shp')
    #hydro_proj = clip.clip_shp(hydro, admin)
    
    #pts3 = hydro_proj.geometry.unary_union
    #points['id'] = np.arange(len(points))

    #def near(point, point2=pts3):
    #    nearest = hydro_proj.geometry == nearest_points(point, point2)[1]
    #    return hydro_proj[nearest].get_values()[0]
    
    #x = unit

    #if x is 'MW':
    #    clusters['hydropower'] = clusters.apply(lambda row: near(row.geometry)[points.columns.get_loc(x)]*1000, axis=1)
    #elif x is 'kW':
    #    clusters['hydropower'] = clusters.apply(lambda row: near(row.geometry)[points.columns.get_loc(x)], axis=1)
    #else:
    #    clusters['hydropower'] = clusters.apply(lambda row: near(row.geometry)[points.columns.get_loc(x)]/1000, axis=1)

    #clusters['hydrodist'] = clusters.geometry.apply(lambda g: points.distance(g).min())
    #clusters['hydropowerFID'] = clusters.apply(lambda row: near(row.geometry)[points.columns.get_loc('id')], axis=1)
    #return clusters

# Conditioning

In [10]:
def conditioning(clusters, workspace, x):
    clusters = clusters.to_crs({ 'init': 'epsg:4326'}) 

    clusters.rename(columns={"NightLight": "NightLights", popunit.value : "Pop", "GridCellAr":"GridCellArea"})

    if "landcover_" in clusters:
        clusters = clusters.rename(columns={"landcover_": "LandCover"})
    else:
        raise Exception('The landcover column is missing, this is a mandatory column rerun cell X')

    if "elevation_" in clusters:
        clusters = clusters.rename(columns={"elevation_": "Elevation"})
    else:
        raise Exception('The elevation column is missing, this is a mandatory column rerun cell X')    

    if "slope_majo" in clusters:
        clusters = clusters.rename(columns={"slope_majo": "Slope"})
    else:
        raise Exception('The slope column is missing, this is a mandatory column rerun cell X')  

    if "ghi_mean" in clusters:
        clusters = clusters.rename(columns={"ghi_mean": "GHI"})
    else:
        raise Exception('The solar column is missing, this is a mandatory column rerun cell X')  

    if "traveltime" in clusters:
        clusters = clusters.rename(columns={"traveltime": "TravelHours"})
    else:
        raise Exception('The traveltime column is missing, this is a mandatory column rerun cell X')

    if "wind_mean" in clusters:
        clusters = clusters.rename(columns={"wind_mean": "WindVel"})
    else:
        raise Exception('The wind speed column is missing, this is a mandatory column rerun cell X')

    if "customdemand" in clusters:
        clusters = clusters.rename(columns={"customdemand": "ResidentialDemandTierCustom"})
    else:
        clusters["ResidentialDemandTierCustom"] = 0

    if "Substation" in clusters:
        clusters = clusters.rename(columns={"Substation": "SubstationDist"})
    else:
        clusters["SubstationDist"] = 99

    if "Existing_H" in clusters:
        clusters = clusters.rename(columns={"Existing_H": "CurrentHVLineDist"})  
    else:
        clusters["CurrentHVLineDist"] = 0    

    if "Planned_HV" in clusters:
        clusters = clusters.rename(columns={"Planned_HV": "PlannedHVLineDist"})
    elif "CurrentHVLineDist" not in clusters:
        clusters["PlannedHVLineDist"] = 0
    else:
        clusters["PlannedHVLineDist"] = clusters["CurrentHVLineDist"]  

    if "Existing_M" in clusters:
        clusters = clusters.rename(columns={"Existing_M": "CurrentMVLineDist"})
    else:
        clusters["CurrentMVLineDist"] = 0

    if "Planned_MV" in clusters:
        clusters = clusters.rename(columns={"Planned_MV": "PlannedMVLineDist"})
    elif "CurrentMVLineDist" not in clusters:
        clusters["PlannedMVLineDist"] = 0
    else:
        clusters["PlannedMVLineDist"] = clusters["CurrentMVLineDist"]    

    if "RoadDist" not in clusters:
        clusters["RoadDist"] = 99

    if "Transforme" in clusters:
        clusters = clusters.rename(columns={"Transforme": "TransformerDist"})
    else:
        clusters["TransformerDist"] = 0

    if "hydropow_1" in clusters:
        clusters = clusters.rename(columns={"hydropow_1": "HydropowerFID"})
    else:
        clusters["HydropowerFID"] = 0

    if "hydrodist" in clusters:
        clusters = clusters.rename(columns={"hydrodist": "HydropowerDist"})
    else:
        clusters["HydropowerDist"] = 99

    if "hydropower" in clusters:
        clusters = clusters.rename(columns={"hydropower": "Hydropower"})
    else:
        clusters["Hydropower"] = 0

    clusters["IsUrban"] = 0
    clusters["PerCapitaDemand"] = 0
    clusters["HealthDemand"] = 0    
    clusters["EducationDemand"] = 0
    clusters["AgriDemand"] = 0
    clusters["CommercialDemand"] = 0
    clusters["Conflict"] = 0
    clusters["ElectrificationOrder"] = 0
    clusters["ResidentialDemandTier1"] = 7.74
    clusters["ResidentialDemandTier2"] = 43.8
    clusters["ResidentialDemandTier3"] = 160.6
    clusters["ResidentialDemandTier4"] = 423.4
    clusters["ResidentialDemandTier5"] = 598.6
    clusters["X_deg"] = clusters.geometry.centroid.x
    clusters["Y_deg"] = clusters.geometry.centroid.y
    clusters.to_file(workspace + r"\output.shp", driver='ESRI Shapefile')
    clusters.to_file(workspace + r"\output.csv", driver='CSV')
    print(datetime.datetime.now())